In [84]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transform
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from tqdm import tqdm


In [76]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
num_epoch = 10
batch_size = 1024
num_classes = 10
lr = 0.001

In [75]:
load_train = datasets.CIFAR10('datasets/', transform=transform.ToTensor())
load_test = datasets.CIFAR10('datasets/',train=False, transform=transform.ToTensor())

train = DataLoader(load_train, batch_size, shuffle=True)
test = DataLoader(load_test, batch_size, shuffle=True)
load_train[0][0].unsqueeze(0).shape

torch.Size([1, 3, 32, 32])

In [63]:
x = torch.randn(1, 3, 224, 224)

In [9]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    
    def forward(self, x):
        return x

In [96]:
model = torchvision.models.vgg19(weights='DEFAULT')

for param in model.parameters():
    param.requires_grad = False


model.avgpool = Identity()
model.classifier = nn.Sequential(
    nn.Linear(512, 100),
    nn.ReLU(),
    nn.Linear(100, 10),
)

model.to(device)
    
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [99]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr)
for epoch in range(num_epoch):
    for x_batch, y_batch in tqdm(train):

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        y_pred = model(x_batch)

        loss = criterion(y_pred, y_batch)
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

100%|██████████| 49/49 [00:10<00:00,  4.56it/s]


In [102]:
def check_accuracy(data, model):
    
    num_samples = 0
    num_correct = 0
    model.eval()

    with torch.no_grad():
        for x, label in data:

            x = x.to(device)
            label = label.to(device)
            y = model(x)

            _, predictions = y.max(1)

            num_correct += (predictions == label).sum()
            num_samples += predictions.shape[0]

    accuracy = num_correct / num_samples * 100
    print(float(accuracy))

In [103]:
check_accuracy(test, model)

60.8599967956543
